In [1]:
import sys
import os
from dotenv import load_dotenv

# 1. 自动处理路径与环境变量
load_dotenv()
current_path = os.getcwd()
if current_path not in sys.path:
    sys.path.append(current_path)

In [2]:
from spark_common import MinIOSparkManager
from setup import LakehouseSetupManager
from bronze import Bronze
from silver import Silver
from gold import Gold

✅ Preloaded GX Suite: reddit_posts_bz_suite


In [3]:
once = True 
processing_time = "5 seconds"

In [4]:
# 1. 初始化 Manager
manager = MinIOSparkManager()

In [5]:
# 2. 启动 Spark
spark = manager.create_session("Reddit_ETL_Pipeline")

:: loading settings :: url = jar:file:/usr/local/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/astro/.ivy2/cache
The jars for the packages stored in: /home/astro/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a7179f1a-2aec-4ed5-b300-44c4e1efe4b8;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.1 in central
	found io.delta#delta-storage;3.2.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 101ms :: artifacts dl 4ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	io.delta#delta-spark_2.12;3.2.1 from central in [default]
	io.delta#delta-storage;3.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.hadoop#

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
try:
    # 4. 初始化 Setup Manager (环境注入)
    # 统一使用 manager 已经从 .env 中解析好的配置，消除 localhost/minio 的冲突
    DBsetup = LakehouseSetupManager(
        spark_session=spark,
        endpoint=manager.endpoint,
        access_key=manager.access_key,
        secret_key=manager.secret_key,
        bucket=manager.bucket
    )

    print(f"🚀 开始全链路 ETL 流程 (Mode: {'Batch' if once else 'Streaming'})")
    print(f"📍 目标存储: {manager.endpoint} / {manager.bucket}")

    # --- 阶段 1: Bronze (Raw -> Delta) ---
    print("\n📦 [STAGE: BRONZE] 正在同步数据至原始层...")
    bronze = Bronze(spark, DBsetup)
    bronze.consume(once, processing_time)

    # --- 阶段 2: Silver (Filter & Cast -> Delta) ---
    print("\n🥈 [STAGE: SILVER] 正在清洗并转换至白银层...")
    silver = Silver(spark, DBsetup)
    silver.upsert(once, processing_time)

    # --- 阶段 3: Gold (CDC & Deduplication -> Delta) ---
    print("\n🥇 [STAGE: GOLD] 正在聚合并去重至黄金层...")
    gold = Gold(spark, DBsetup)
    gold.upsert(once, processing_time)

    print("\n✨ 所有层级处理完成！")

except Exception as e:
    print(f"\n❌ 流程执行失败: {str(e)}")
    raise

finally:
    # 5. 确保 Spark 资源被释放
    print("🛑 正在关闭 Spark Session...")
    spark.stop()

🚀 开始全链路 ETL 流程 (Mode: Batch)
📍 目标存储: http://minio:9000 / reddit-warehouse

📦 [STAGE: BRONZE] 正在同步数据至原始层...
📂 Bronze 数据基准路径: /usr/local/airflow/include/data
💾 Checkpoint 根路径: s3a://reddit-warehouse/_checkpoints

🚀 Starting bronze layer consumption ...


26/01/15 00:00:10 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


26/01/15 00:00:11 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


26/01/15 00:00:12 WARN CacheManager: Asked to cache already cached data.


🚀 Batch 2: Running GX for reddit_posts_bz...


...

26/01/15 00:00:16 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


26/01/15 00:00:18 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


✅ Silver 层更新完成，耗时: 8 秒

🥇 [STAGE: GOLD] 正在聚合并去重至黄金层...
💾 Gold Checkpoint 根路径: s3a://reddit-warehouse/_checkpoints

🚀 执行 Gold 层 Upsert (自包含模式) ...


26/01/15 00:00:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


✅ 完成 Gold 层加工，耗时 13 秒

✨ 所有层级处理完成！
🛑 正在关闭 Spark Session...
